# Побудова оцінок та довірчих інтервалів

Будуємо довірчі інтервали для `n = 100, 10_000, 1_000_000`
Рівень довіри = `0,99` (рівень значущості = `0.01`

#### Довірчі інтервали для м.с. та дисперсії у випадку нормально розподіленої генеральної сукупності.
##### Перша частина леми 1. (Пункт А):
Вибіркове математичне сподівання: $\hat{a}_{n}=\frac {1}{n}\sum^{n}_{k=1}X_k$
Вибіркова дисперсія: $\hat{\sigma}^2_{n}=\frac {1}{n}\sum^{n}_{k=1}(X_k-\hat{a}_{n})^2$
Дов інтервал $a \in a_n \pm \frac{z_{\gamma}  \sigma_n}{\sqrt{n}}$
К-ть ітерацій: $n$

In [1]:
import math

import numpy as np
from scipy.stats import chi2, t, norm

In [2]:
def calc_sample_variance(samples, sample_mean: float):
    squared_variance = 0.
    for sample in samples:
        squared_variance += (sample - sample_mean) ** 2

    squared_variance /= (len(samples) - 1)
    return math.sqrt(squared_variance)


def calc_sample_mean(samples):
    mean = 0.
    for sample in samples:
        mean += sample
    return mean / len(samples)


def calc_s_squared(samples, sample_mean):
    s_squared = 0
    for sample in samples:
        s_squared += (sample - sample_mean) ** 2

    s_squared /= (len(samples) - 1)
    return s_squared


# generating samples with help of N(0,1) distribution
def gen_samples(size: int):
    samples = size * [None]
    for i in range(size):
        samples[i] = np.random.normal(0, 1)
    return samples

In [3]:
n_arr = [100, 10_000, 1_000_000]  # for calculating Confidence Intervals
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m

samples_list = len(n_arr) * [None]
alfa = 0.01
gama = 1 - alfa
z_gama = 2.575  # norm distro 0.99 quantile

Q_m = lambda q, n: np.sum(q) / n
# calculating variance in integral methods
# незміщена?
calc_variance = lambda q, n, Q: (1 / (n - 1)) * (np.sum([qi ** 2 for qi in q]) - n * Q ** 2)

eta = lambda x: 1 / x - 1
xi_i = lambda x: -1 * np.log(x)

# calculating 0.99 quantiles
print(f"samples_num_ci:{n_arr}\nm_arr:{m_arr}")

samples_num_ci:[100, 10000, 1000000]
m_arr:[1, 10, 100, 1000, 10000]


In [4]:
# calculating Confidence Interval for the EXPECTATION(sigma^2) under the assumption:
# - observed random value ~ N(0,1)
# - sample variance is unknown
def calculate_ci_1(samples):
    sample_mean = calc_sample_mean(samples)
    n_sq_root = math.sqrt(len(samples))
    s_squared = calc_s_squared(samples, sample_mean)

    g = t.ppf((1 + gama) / 2, df=len(samples) - 1)
    return [sample_mean - math.sqrt(s_squared) * g / n_sq_root, sample_mean + math.sqrt(s_squared) * g / n_sq_root]


# calculating Confidence Interval for the EXPECTATION under assumption:
# - observed value distribution is unknown
# asymptotic confidence interval
def calculate_ci_2(samples, gama):
    sample_mean = calc_sample_mean(samples)
    sample_variance = calc_sample_variance(samples, sample_mean)

    n_sq_root = math.sqrt(len(samples))

    c = norm.ppf((1 + gama) / 2)
    return [sample_mean - sample_variance * c / n_sq_root, sample_mean + sample_variance * c / n_sq_root]


# calculating Confidence Interval for the variance under assumption:
# - observed random value ~ N(0,1)
def calculate_ci_3(samples, gama):
    sample_mean = calc_sample_mean(samples)
    g_1 = chi2.ppf((1 - gama) / 2, df=len(samples) - 1)
    g_2 = chi2.ppf((1 + gama) / 2, df=len(samples) - 1)

    squared_difference = 0
    for sample in samples:
        squared_difference = (sample - sample_mean) ** 2

    return [squared_difference / g_2, squared_difference / g_1]

In [5]:
for i, sample_size in enumerate(n_arr):
    samples_list[i] = gen_samples(sample_size)
    sample_mean = calc_sample_mean(samples_list[i])
    sample_variance = calc_sample_variance(samples_list[i], sample_mean)

    ci_1 = calculate_ci_1(samples_list[i])
    print('\n[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_1[0]} ; {ci_1[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_1[1] - ci_1[0]}')

    ci_2 = calculate_ci_2(samples_list[i], gama)
    print('\n[2] Довірчий інтервал для мат. спод. M при невідомому розподілі')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркове математичне сподівання: {sample_mean}')
    print(f'Довірчий інтервал:[{ci_2[0]} ; {ci_2[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_2[1] - ci_2[0]}')

    ci_3 = calculate_ci_3(samples_list[i], gama)
    print('\n[3] Довірчий інтервал для дисперсії')
    print(f'Кількість спостережень: n = {sample_size}')
    print(f'Вибіркова дисперсія = {sample_variance}')
    print(f'Довірчий інтервал:[{ci_3[0]} ; {ci_3[1]}]')
    print(f'Довжина довірчого інтервалу: {ci_3[1] - ci_3[0]}')


[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 100
Вибіркове математичне сподівання: -0.014112862771841478
Довірчий інтервал:[-0.27835801756618267 ; 0.2501322920224997]
Довжина довірчого інтервалу: 0.5284903095886824

[2] Довірчий інтервал для мат. спод. M при невідомому розподілі
Кількість спостережень: n = 100
Вибіркове математичне сподівання: -0.014112862771841478
Довірчий інтервал:[-0.2732695026521448 ; 0.2450437771084618]
Довжина довірчого інтервалу: 0.5183132797606066

[3] Довірчий інтервал для дисперсії
Кількість спостережень: n = 100
Вибіркова дисперсія = 1.0061095256709949
Довірчий інтервал:[2.0030059247709372e-06 ; 4.1857000444384354e-06]
Довжина довірчого інтервалу: 2.182694119667498e-06

[1] Довірчий інтервал для мат. спод. M при невідомій дисперсії
Кількість спостережень: n = 10000
Вибіркове математичне сподівання: 0.010642912384757076
Довірчий інтервал:[-0.01510604923341709 ; 0.03639187400293124]
Довжина довірчого інтервалу: 0.

# Метод 1
Будемо обраховувати інтеграл за допомогою методу Монте-Карло:
$$Q_m = \bf{M}(I(\eta > \xi_{1} + \ldots + \xi_m))$$

(варто підмітити, що із збільшенням m значення інтеграла зменшується у $\sqrt{m}$ разів)

In [6]:
q_1 = lambda eta, samples: 1 if eta > np.sum(samples) else 0
m_arr = [1, 10, 100, 1_000]  # for calculating Q_m
n_0 = 10000  #

for m in m_arr:
    indicators = []

    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_1(eta(np.random.random(1)), samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(z_gama * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6901
Вибіркова дисперсія: 0.21388337833783375
Асимптотичний довірчий інтервал: [0.6781874263907887, 0.7020125736092114]
Кількість спостережень: 10000
N*: 29778.817433982196

Кількість зразків: 10
Оцінка Q: 0.1711
Вибіркова дисперсія: 0.14183897389738975
Асимптотичний довірчий інтервал: [0.16139903651277826, 0.18080096348722174]
Кількість спостережень: 10000
N*: 321255.0981866159

Кількість зразків: 100
Оцінка Q: 0.0175
Вибіркова дисперсія: 0.017195469546954695
Асимптотичний довірчий інтервал: [0.014122276470016662, 0.020877723529983343]
Кількість спостережень: 10000
N*: 3722994.6208906597

Кількість зразків: 1000
Оцінка Q: 0.0028
Вибіркова дисперсія: 0.0027924392439243927
Асимптотичний довірчий інтервал: [0.0014388407276122613, 0.0041611592723877384]
Кількість спостережень: 10000
N*: 23616859.007329307



# Метод 2

$$Q_m = \bf{M} \frac{1}{1+\xi_1 + \ldots + \xi_m}$$


In [7]:

m_arr = [1, 10, 100, 1_000, 10_000]  # for calculating Q_m
n_0 = 100
q_2 = lambda samples: 1 / (1 + np.sum(samples))

for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))
    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)
    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6831284608009885
Вибіркова дисперсія: 0.017179828773842245
Асимптотичний довірчий інтервал: [0.6493665906605931, 0.7168903309413842]
Кількість спостережень: 100
N*: 2441.00426835169

Кількість зразків: 10
Оцінка Q: 0.17138982443397455
Вибіркова дисперсія: 0.0008435661919523854
Асимптотичний довірчий інтервал: [0.16390853145767612, 0.17887111741027292]
Кількість спостережень: 100
N*: 1904.1605571126988

Кількість зразків: 100
Оцінка Q: 0.019660097856589353
Вибіркова дисперсія: 1.2401263380989286e-06
Асимптотичний довірчий інтервал: [0.01937325103291007, 0.019946944680268634]
Кількість спостережень: 100
N*: 212.7399474679719

Кількість зразків: 1000
Оцінка Q: 0.001996764880835993
Вибіркова дисперсія: 1.6707865261333337e-09
Асимптотичний довірчий інтервал: [0.001986236112701366, 0.0020072936489706206]
Кількість спостережень: 100
N*: 27.78571467133126

Кількість зразків: 10000
Оцінка Q: 0.0001998678189002261
Вибіркова дисперсія: 1.6084685038629628e-12
Асимп

# Метод 4

$$
Q_m = \frac{1}{m-1} \bf{M} \frac{\xi_1 + \ldots + \xi_{m-1}}{1+\xi_1 + \ldots + \xi_{m-1}}
$$

In [8]:
m_arr = [1, 10, 100, 1_000, 10_000]  # for calculation Q_m
n_0 = 5
q_4 = lambda m, x, samples: (1 / (m - 1)) * ((np.sum(samples)) / (1 + np.sum(samples)))

for m in m_arr:
    indicators = []
    for j in range(n_0):
        samples = np.random.random(m)
        indicators.append(q_2(samples))

    Q = Q_m(indicators, len(indicators))
    sigma = calc_variance(indicators, len(indicators), Q)

    print(f"Кількість зразків: {m}")
    print(f"Оцінка Q: {Q}")
    print(f"Вибіркова дисперсія: {sigma}")
    print(f"Асимптотичний довірчий інтервал: {calculate_ci_2(indicators, gama)}")
    print(f"Кількість спостережень: {n_0}")
    print(f"N*: {(2.575 * 100) ** 2 * sigma / (Q ** 2)}\n")

Кількість зразків: 1
Оцінка Q: 0.6906684924118055
Вибіркова дисперсія: 0.033639057298284536
Асимптотичний довірчий інтервал: [0.47939063416350436, 0.9019463506601066]
Кількість спостережень: 5
N*: 4675.833031203439

Кількість зразків: 10
Оцінка Q: 0.14677748785120465
Вибіркова дисперсія: 0.00031842364836962705
Асимптотичний довірчий інтервал: [0.1262216712425051, 0.1673333044599042]
Кількість спостережень: 5
N*: 980.0334072561062

Кількість зразків: 100
Оцінка Q: 0.02017893724091381
Вибіркова дисперсія: 9.574382510026515e-07
Асимптотичний довірчий інтервал: [0.01905177231977184, 0.021306102162055782]
Кількість спостережень: 5
N*: 155.90809379011128

Кількість зразків: 1000
Оцінка Q: 0.001995570588493485
Вибіркова дисперсія: 3.907945610043713e-09
Асимптотичний довірчий інтервал: [0.0019235583464161012, 0.002067582830570869]
Кількість спостережень: 5
N*: 65.06819932788676

Кількість зразків: 10000
Оцінка Q: 0.0001999279720467157
Вибіркова дисперсія: 1.6563953230731117e-12
Асимптотичний д